### 爬取日本國土交通取氣象廳的開花日資料
### 以下連結為 さくらの開花日(1961-1970年）
### https://www.data.jma.go.jp/sakura/data/sakura003_01.html

In [2]:
import requests
from bs4 import BeautifulSoup

# 目標網址
url = 'https://www.data.jma.go.jp/sakura/data/sakura003_01.html'

# 發送請求
response = requests.get(url)
response.encoding = 'utf-8'

# 確認請求成功
if response.status_code == 200:
    # 解析 HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 找出所有 <pre> 標籤（資料都在這裡面）
    pre_tags = soup.find_all('pre')
    
    # 印出每個 <pre> 裡的文字內容
    for i, pre in enumerate(pre_tags):
        print(f"\n--- 第 {i+1} 個 <pre> 區塊 ---\n")
        print(pre.get_text())
else:
    print(f"無法取得網頁，狀態碼：{response.status_code}")


--- 第 1 個 <pre> 區塊 ---



地点名　     1961   1962   1963   1964   1965   1966   1967   1968   1969   1970   平年値   代替種目
            月 日  月 日  月 日  月 日  月 日  月 日  月 日  月 日  月 日  月 日   月 日

稚内     *   5 11   5 10   5 12   5 16   5 18      -      -      -      -      -    5 13    えぞやまざくら
留萌         5  7   5  9   5 13   5  8   5 18   5 16   5 11   5  8   5 16   5  9       -    えぞやまざくら
旭川     *   5  8   5  8   5 10   5 12   5 15   5 14   5  5   5  2   5  6   5  7    5  4    えぞやまざくら
網走     *   5  8      -   5 13      -   5 17   5 18   5 10   5  6   5  9   5  8    5 10    えぞやまざくら
札幌     *   5  4   4 30   5  2   5  4   5 13   5  3   5  1   4 26   5  5   5  3    5  1    
岩見沢       5  4   5  3   5  8   5  7   5 16   5 12   5  5   5  3   5 10   5  9       -    えぞやまざくら
帯広     *   5  5   5  4   5  7   5  8   5 15   5 13   5  3   5  2   5  7   5 10    5  2    えぞやまざくら
釧路     *      -      -      -      -      -      -      -      -      -      -    5 16    えぞやまざくら
根室         5 18   5 15   5 18   5 22   

In [3]:
import requests
from bs4 import BeautifulSoup
import csv

# 對應網頁與年份範圍
pages = {
    "sakura003_01.html": list(range(1961, 1971)),
    "sakura003_02.html": list(range(1971, 1981)),
    "sakura003_03.html": list(range(1981, 1991)),
    "sakura003_04.html": list(range(1991, 2001)),
    "sakura003_05.html": list(range(2001, 2011)),
    "sakura003_06.html": list(range(2011, 2021)),
    "sakura003_07.html": list(range(2021, 2025))
}

base_url = "https://www.data.jma.go.jp/sakura/data/"

# 準備存成多列的 CSV（每頁一列）
rows = []

for page, years in pages.items():
    url = base_url + page
    response = requests.get(url)
    response.encoding = 'utf-8'

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        pre = soup.find('pre')

        lines = pre.get_text().splitlines()

        tokyo_line = ""
        for line in lines:
            if line.strip().startswith("東京"):
                tokyo_line = line.strip()
                break

        if tokyo_line:
            rows.append([f"{years[0]}–{years[-1]}", tokyo_line])
            print(f"✅ 已抓取 {page} 的東京資料")
        else:
            rows.append([f"{years[0]}–{years[-1]}", "找不到東京資料"])
            print(f"⚠️ 無法找到 {page} 中的東京資料")
    else:
        rows.append([f"{years[0]}–{years[-1]}", f"無法連線（狀態碼：{response.status_code})"])
        print(f"❌ 抓取失敗：{url}")

# 寫入 CSV
with open('sakura_blossom_date_from_1961_to_2024.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['年份區間', '開花日'])
    writer.writerows(rows)

print("\n🌸 所有頁面已抓取並寫入 sakura_blossom_date_from_1961_to_2024.csv")

✅ 已抓取 sakura003_01.html 的東京資料
✅ 已抓取 sakura003_02.html 的東京資料
✅ 已抓取 sakura003_03.html 的東京資料
✅ 已抓取 sakura003_04.html 的東京資料
✅ 已抓取 sakura003_05.html 的東京資料
✅ 已抓取 sakura003_06.html 的東京資料
✅ 已抓取 sakura003_07.html 的東京資料

🌸 所有頁面已抓取並寫入 sakura_blossom_date_from_1961_to_2024.csv
